<a href="https://colab.research.google.com/github/hieubkset/Colab-Notebooks/blob/master/text_classification_with_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phân loại văn bản**

Một ví dụ về bài toán phân loại văn bản trên tập dữ liệu [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb).

Mục tiêu:

+ Có cái nhìn tổng quan về [TensorFlow Hub](https://www.tensorflow.org/hub) và [TensorFlow datasets](https://www.tensorflow.org/datasets).
+ Hiểu cơ bản việc sử dụng transfer learning để chuyển đổi từ text sang embedding vector.

## **Giới thiệu IMDB dataset**

[IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) bao gồm review (đánh giá) của 50,000 bộ phim từ [Internet Movie Database](https://www.imdb.com/). Trong đó, 25,000 review cho training và 25,000 reivew còn lại cho testing. Mỗi review là một câu bình luận về bộ phim và được gán một trong 2 nhãn: positive (tích cực) hoặc negative (tiêu cực). Số lượng nhãn positive và negative là chia đều trong cả tập training và testing. 

## **Chương trình**

### **1. Khai báo các thư viện**

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE" )

Version:  2.1.0
Hub version:  0.7.0
GPU is available


### **2. Chuẩn bị data**

Ở các bài trước, chúng ta đã học cách load data với **tf.keras.datasets**. Ở bài này, chúng ta sẽ làm quen với một kho data khác to hơn là [Tensorflow Datasets](https://www.tensorflow.org/datasets). 

**Tensorflow Datasets** cung cấp khoảng 29 bộ dataset như: MNIST, Street View House Numbers, the 1 Billion Word Language Model Benchmark, v.v... và được cập nhật thường xuyên.

Tất cả các dataset trong **Tensorflow Datasets** là một thể hiển của  [tf.data.Datasets](https://www.tensorflow.org/api_docs/python/tf/data/Dataset). Hiểu một cách đơn giản là các dataset này đã chuẩn hóa nên rất dễ sử dụng và cho hiệu năng cao.

Dưới đây là một ví dụ load **IMDB dataset** với **Tensorflow Datasets** (tfds):

In [0]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

** Explore data**

Chúng ta sẽ xem định dạng của dữ liệu trước khi tiến hành build model:

In [0]:
train_examples_batch, train_lables_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [0]:
train_lables_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

Label gồm 2 giá trị: 1 tương ứng với positive và 0 tương ứng với negative.

### **3. Build model**

Để xử lý dữ liệu đầu vào là các review dưới dạng text có chiều dài bất kỳ, chúng ta sẽ sử dụng một pre-trained text embedding model để chuyển đầu vào từ text sang embedding vector (có chiều dài cố định).

Cụ thể, chúng ta sử dụng [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) từ [TensorFlow Hub](https://www.tensorflow.org/hub).

**TensorFlow Hub** là nơi cung cấp các pre-trained model (các model được train trên các tập dataset lớn, nhiều data). Chúng ta có thể dùng toàn bộ hoặc một phần của pre-trained model để tiến hành training trên dataset của chúng ta (thường ít data). Quá trình này gọi là transfer learning.

**TensorFlow Hub** hỗ trợ việc sử dụng pre-trained model như một keras layer. Ví dụ: 


In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

*Dữ liệu được chuyển từ text sang embedding vector có 20 chiều.*

**Định nghĩ model:**
+ pre-trained model được sử dụng như input layer chuyển đổi đầu vào từ text sang embedding vector có số chiều cố định là 20.
+ Dense layer với 16 hidden unit
+ Output layer với 1 unit. Vì bài toán chỉ có 2 class nên ta chỉ cần một output cho biết xác suất ứng với possitive.

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### **4. Khai báo Optimizer và Loss function**
Chúng ta vẫn sử dụng Adam Optimizer :) Tuy nhiên, vì chỉ có 2 class nên ta dùng **BinaryCrossentropy**.

In [0]:
model.compile(optimizer='adam', 
              loss= tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=['accuracy'])

### **5. Tiến hành training**

Train model trong 20 epoch với batch size là 512. Trong quá trình training, các thông số như loss và accuracy trên tập validation sẽ được lưu lại (*chúng ta sẽ tìm hiểu cách hiển thị các thông số này ở bài sau*).


In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 159ms/step - loss: 1.5774 - accuracy: 0.5058 - val_loss: 0.8034 - val_accuracy: 0.5157
Epoch 2/20
30/30 [==============================] - 4s 133ms/step - loss: 0.7509 - accuracy: 0.5371 - val_loss: 0.7239 - val_accuracy: 0.5523
Epoch 3/20
30/30 [==============================] - 4s 141ms/step - loss: 0.6924 - accuracy: 0.5746 - val_loss: 0.6815 - val_accuracy: 0.5860
Epoch 4/20
30/30 [==============================] - 4s 142ms/step - loss: 0.6478 - accuracy: 0.6129 - val_loss: 0.6408 - val_accuracy: 0.6297
Epoch 5/20
30/30 [==============================] - 4s 148ms/step - loss: 0.6057 - accuracy: 0.6559 - val_loss: 0.5995 - val_accuracy: 0.6595
Epoch 6/20
30/30 [==============================] - 4s 145ms/step - loss: 0.5677 - accuracy: 0.6899 - val_loss: 0.5681 - val_accuracy: 0.6907
Epoch 7/20
30/30 [==============================] - 5s 159ms/step - loss: 0.5368 - accuracy: 0.7156 - val_loss: 0.5405 - val_accuracy: 0.7063
Epoch 

**6. Testing**

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.333
accuracy: 0.850


Với 20 epoch, model cho accuracy trên tập test khoảng 87%. Với cách tiếp cận nâng cao hơn, accuracy có thể lên tới 95%.